In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [4]:
!pip install openpyxl

In [5]:
train_df = pd.read_excel(
    'https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true',
    sheet_name='train')
test_df = pd.read_excel(
    'https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true',
    sheet_name='test')

In [49]:
x_train = train_df.drop(['가격'], axis=1)
x_test = test_df.drop(['가격'], axis=1)
y_train = train_df[['가격']]
y_test = test_df[['가격']]

In [8]:
print(x_train.head())

     년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1  2015  준중형  12.3  204  27.0  가솔린      0  1591  1300  자동
2  2015   소형  15.0  100  13.6  가솔린      0  1368  1035  수동
3  2014   소형  14.0  140  17.0  가솔린      0  1591  1090  자동
4  2015   대형   9.6  175  46.0   디젤      0  2497  1990  자동


In [9]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['종류', '연료', '변속기']),
    remainder='passthrough')
transformer.fit(x_train)
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [10]:
pd.get_dummies(train_df)

,가격,년식,연비,마력,토크,하이브리드,배기량,중량,종류_대형,종류_소형,종류_준중형,종류_중형,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,1885,2015,11.8,172,21.0,0,1999,1300,0,0,1,0,0,1,0,0,1
1,2190,2015,12.3,204,27.0,0,1591,1300,0,0,1,0,0,1,0,0,1
2,1135,2015,15.0,100,13.6,0,1368,1035,0,1,0,0,0,1,0,1,0
3,1645,2014,14.0,140,17.0,0,1591,1090,0,1,0,0,0,1,0,0,1
4,1960,2015,9.6,175,46.0,0,2497,1990,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,3802,2015,8.5,290,34.8,0,3342,1901,0,0,0,1,0,1,0,0,1
67,1270,2012,13.3,108,13.9,0,1396,1040,0,1,0,0,0,1,0,0,1
68,2430,2015,12.8,186,41.0,0,1995,1665,0,0,1,0,0,0,1,0,1
69,2870,2015,17.7,156,19.3,1,1999,1585,0,0,0,1,0,1,0,0,1


In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
# minmax는 더미변수 값의 0과 1에 변형이 없어서
# 별도 변수 분리없이 모두에 적용

scaler = MinMaxScaler()
scaler_x_train = scaler.fit_transform(x_train)
scaler_x_test = scaler.transform(x_test)

In [15]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(scaler_x_train, y_train)
model.coef_, model.intercept_

(array([[-538.07573372,  463.8614926 ,   10.9448916 ,   63.26934952,
          288.56121264, -212.79830412,  -75.76290851, -161.29147786,
          161.29147786,  140.42708513, 1666.18511725, 3109.14675071,
         -301.08509852,  325.68069979, 6567.11718155,  150.92399575]]),
 array([-408.18794177]))

In [18]:
model.score(scaler_x_test, y_test)

0.7739730315244938

In [67]:
# 변수 중요도는 coef_로 확인한다

coeffDf = pd.DataFrame({
    'key': pd.get_dummies(train_df.drop(['가격'], axis=1)).columns,
    'coeff': model.coef_[0]
})

coeffDf.sort_values('coeff')

,key,coeff
0,년식,-538.075734
1,연비,-301.085099
2,마력,-212.798304
3,토크,-161.291478
4,하이브리드,-75.762909
5,배기량,10.944892
6,중량,63.269350
7,종류_대형,140.427085
8,종류_소형,150.923996
9,종류_준중형,161.291478


In [73]:
# OLS (최소제곱법: )
import statsmodels.api as sm

x2 = sm.add_constant(scaler_x_train)
model_OLS = sm.OLS(train_df['가격'], x2)
result = model_OLS.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     가격   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     33.58
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           6.27e-22
Time:                        17:09:56   Log-Likelihood:                -540.77
No. Observations:                  71   AIC:                             1110.
Df Residuals:                      57   BIC:                             1141.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -195.9302    341.907     -0.573      0.569    -880.587     488.727
x1          -587.0583    283.102     -2.074      0.043   -1153.960     -20.157
x2           414.8789    253.318      1.638      0.107     -92.382     922.140
x3           -38.0377    162.546     -0.234      0.816    -363.531     287.455
x4            14.2868    240.580      0.059      0.953    -467.466     496.039
x5           223.2511    222.444      1.004      0.320    -222.184     668.687
x6          -278.1084    190.909     -1.457      0.151    -660.397     104.180
x7          -141.0730    306.451     -0.460      0.647    -754.730     472.584
x8          -259.2566    210.255     -1.233      0.223    -680.284     161.771
x9            63.3264    169.563      0.373      0.710    -276.218     402.870
x10          140.4271    336.952      0.417      0.678    -534.308     815.162
x11         1666.1851    759.186      2.195      0.032     145.942    3186.428
x12         3109.1468   1353.780      2.297      0.025     398.250    5820.044
x13         -301.0851    604.272     -0.498      0.620   -1511.120     908.950
x14          325.6807    502.303      0.648      0.519    -680.164    1331.525
x15         6567.1172   1517.244      4.328      0.000    3528.888    9605.346
x16          150.9240   1059.427      0.142      0.887   -1970.543    2272.391
==============================================================================
Omnibus:                       25.864   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.139
Skew:                           0.896   Prob(JB):                     6.62e-23
Kurtosis:                       8.596   Cond. No.                     3.64e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.06e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [74]:
model.coef_

array([[-538.07573372, -301.08509852, -212.79830412, -161.29147786,
         -75.76290851,   10.9448916 ,   63.26934952,  140.42708513,
         150.92399575,  161.29147786,  288.56121264,  325.68069979,
         463.8614926 , 1666.18511725, 3109.14675071, 6567.11718155]])

In [70]:
scaler_x_train.shape

(71, 16)

In [76]:
result.params # 선형회귀의 coef_와 같음

const    -195.930212
x1       -587.058287
x2        414.878940
x3        -38.037661
x4         14.286797
x5        223.251142
x6       -278.108375
x7       -141.072979
x8       -259.256584
x9         63.326372
x10       140.427085
x11      1666.185117
x12      3109.146751
x13      -301.085099
x14       325.680700
x15      6567.117182
x16       150.923996
dtype: float64

In [77]:
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF Factor'] = [
    variance_inflation_factor(x_data2.valeus, i)
    for i in range(x_data2.shape[1])
]
vif['features'] = x_data2.columns
vif

NameError: name 'x_data2' is not defined